Before getting started:
1. put all the .py files in a Utils folder
2. in config.py make sure the paths work with your setup

In [ ]:
import polars as pl
import importlib

import Utils.config as configs
import Utils.grid_sampling as grid_samp
import Utils.grid_generation as grid_gen
import Utils.force_frame as force_frame
import Utils.path_optimisation as path_opt
import Utils.visualisation as vis
import Utils.book_keeping as bk

importlib.reload(configs)
importlib.reload(grid_samp)
importlib.reload(grid_gen)
importlib.reload(force_frame)
importlib.reload(path_opt)
importlib.reload(vis)
importlib.reload(bk)

In [ ]:
config = configs.config.modelV1(3300,n_lanes=11)
grid = grid_gen.Grid(config, configs.folder_config.TRACK_FOLDER + "\\TrackData.csv")
df = grid.data_generation()
costs_df = force_frame.Forces.add_Forces_and_constraints(config, df)
traj,V = path_opt.Optimisation.run_stack(config, costs_df, show_plots=True)